In [15]:
from gp3.inference import Laplace
from gp3.kernels import RBF
from gp3.likelihoods import Poisson
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
from plotly import tools
from IPython.display import display
init_notebook_mode(connected=True)
import numpy as np

We're interested in modeling the spatial dynamics of donations to political campaigns. Below, we see the number of donations to Hilary Clinton in the most recent election cycle. We have divided the map of the U.S. into a partial grid.

In [16]:
X_grid_fec = np.genfromtxt('fec_data/X_grid.csv', delimiter = ',')
fec_counts = np.genfromtxt('fec_data/y_hillary.csv', delimiter = ',')
obs_idx_fec = np.genfromtxt('fec_data/obs_idx.csv', delimiter = ',', dtype = np.int32)
trace_fec_counts = go.Scatter3d(x = X_grid_fec[obs_idx_fec,1], y = X_grid_fec[obs_idx_fec,0],
                                z=fec_counts, mode = 'markers', marker=dict(size = 2,))
iplot([trace_fec_counts])

We run inference using gp3's Laplace implementation. Note that we can exploit full grid structure by assuming the map fills a part of a complete grid.

In [17]:
mu = np.ones(X_grid_fec.shape[0])*np.log(np.mean(fec_counts))
kern = RBF(10., 1., 1e-2)
likelihood = Poisson()

inf_svi = Laplace(X_grid_fec, fec_counts, kern, Poisson(), mu = mu, obs_idx = obs_idx_fec)
inf_svi.run(20)

Objective: -21152412.41 | Step Size: 0.00:   0%|          | 0/20 [00:00<?, ?it/s]


In [18]:
iplot([go.Scatter3d(x = X_grid_fec[obs_idx_fec,1], y = X_grid_fec[obs_idx_fec,0], z=inf_svi.f_pred[obs_idx_fec], mode = 'markers', marker=dict(size = 2,), name = 'SVI posterior mean')])